In [6]:
import pandas as pd
import pandasql as ps

In [2]:
# URL_DATA='https://cloud.minsa.gob.pe/s/ZgXoXqK2KLjRLxD/download'
# df=pd.read_html(URL_DATA)

df=pd.read_csv('../vacunas_covid.csv')
print(df)

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210407,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210407,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210318,2,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
2,20210407,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210407,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210312,2,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210407,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
...,...,...,...,...,...,...,...,...,...,...,...,...
948896,20210407,67acfb123e8b643072ead029ca105173,PERSONAL DE SALUD,28.0,MASCULINO,20210407,1,SINOPHARM,LIMA ESTE,LIMA,LIMA,ATE
948897,20210407,b833019e4c3016f5393a81e9c28e5e25,TRABAJADOR Ó PERSONAL DE LIMPIEZA,29.0,MASCULINO,20210407,1,SINOPHARM,LIMA ESTE,LIMA,LIMA,ATE
948898,20210407,42c4888119347ddf9ea23203877bf9d7,TRABAJADOR Ó PERSONAL DE LIMPIEZA,42.0,FEMENINO,20210407,1,SINOPHARM,LIMA ESTE,LIMA,LIMA,ATE
948899,20210407,da9849ec7729a432f5a0afa913dd504c,PERSONAL DE SALUD,37.0,FEMENINO,20210407,1,SINOPHARM,LIMA ESTE,LIMA,LIMA,ATE


In [3]:
# Delete all not wanted columns
df.drop(['UUID', 'GRUPO_RIESGO', 'EDAD','SEXO','FABRICANTE','DIRESA','PROVINCIA','DISTRITO'], axis=1, inplace=True)

In [4]:
print(df)

,FECHA_CORTE,FECHA_VACUNACION,DOSIS,DEPARTAMENTO
0,20210407,20210225,1,LAMBAYEQUE
1,20210407,20210318,2,LAMBAYEQUE
2,20210407,20210219,1,LAMBAYEQUE
3,20210407,20210312,2,LAMBAYEQUE
4,20210407,20210220,1,LAMBAYEQUE
...,...,...,...,...
948896,20210407,20210407,1,LIMA
948897,20210407,20210407,1,LIMA
948898,20210407,20210407,1,LIMA
948899,20210407,20210407,1,LIMA


In [13]:
# Please write the full query on one line
# ACUMULADO DOSIS 1
query_dosis1="""select FECHA_CORTE, FECHA_VACUNACION, DOSIS, count (rowid) as TOTAL, sum(count (rowid)) over (order by FECHA_VACUNACION) as ACUMULADO from df where "DOSIS" = 1 group by FECHA_VACUNACION """
result_query_dosis1=ps.sqldf(query_dosis1,locals())
print(ps.sqldf(query_dosis1,locals()))

    FECHA_CORTE  FECHA_VACUNACION  DOSIS  TOTAL  ACUMULADO
0      20210407          20210209      1   3837       3837
1      20210407          20210210      1  16575      20412
2      20210407          20210211      1  33849      54261
3      20210407          20210212      1  32356      86617
4      20210407          20210213      1  18922     105539
5      20210407          20210214      1   5194     110733
6      20210407          20210215      1  11553     122286
7      20210407          20210216      1   7326     129612
8      20210407          20210217      1   3730     133342
9      20210407          20210218      1   6886     140228
10     20210407          20210219      1  29433     169661
11     20210407          20210220      1  25553     195214
12     20210407          20210221      1   4232     199446
13     20210407          20210222      1  27658     227104
14     20210407          20210223      1  26511     253615
15     20210407          20210224      1  22044     2756

In [14]:
# Please write the full query on one line
# ACUMULADO DOSIS 2
query_dosis2="""select FECHA_CORTE, FECHA_VACUNACION, DOSIS, count (rowid) as TOTAL, sum(count (rowid)) over (order by FECHA_VACUNACION) as ACUMULADO from df where "DOSIS" = 2 group by FECHA_VACUNACION"""
result_query_dosis2=ps.sqldf(query_dosis2,locals())
print(ps.sqldf(query_dosis2,locals()))

    FECHA_CORTE  FECHA_VACUNACION  DOSIS  TOTAL  ACUMULADO
0      20210407          20210302      2   1623       1623
1      20210407          20210303      2  10991      12614
2      20210407          20210304      2  25894      38508
3      20210407          20210305      2  28817      67325
4      20210407          20210306      2  14015      81340
5      20210407          20210307      2   1970      83310
6      20210407          20210308      2  10461      93771
7      20210407          20210309      2   6091      99862
8      20210407          20210310      2   4455     104317
9      20210407          20210311      2   4648     108965
10     20210407          20210312      2  14276     123241
11     20210407          20210313      2  25390     148631
12     20210407          20210314      2   2536     151167
13     20210407          20210315      2  23680     174847
14     20210407          20210316      2  21079     195926
15     20210407          20210317      2  20169     2160

In [15]:
# Please write the full query on one line
# ACUMULADO DEPARTAMENTOS
query_departamentos="""select FECHA_CORTE, DEPARTAMENTO, count (case when DOSIS = 1 then 1 end) as DOSIS1, count (case when DOSIS = 2 then 1 end) as DOSIS2 from df group by DEPARTAMENTO"""
result_query_departamentos=ps.sqldf(query_departamentos,locals())
print(ps.sqldf(query_departamentos,locals()))

    FECHA_CORTE   DEPARTAMENTO  DOSIS1  DOSIS2
0      20210407       AMAZONAS    5221    4736
1      20210407         ANCASH   16561   10246
2      20210407       APURIMAC    7668    6552
3      20210407       AREQUIPA   31290   16818
4      20210407       AYACUCHO    9523    7756
5      20210407      CAJAMARCA   12851   11155
6      20210407         CALLAO   53790   14835
7      20210407          CUSCO   14281   11644
8      20210407   HUANCAVELICA    5733    4993
9      20210407        HUANUCO    9489    7879
10     20210407            ICA   12883    9626
11     20210407          JUNIN   17101   13560
12     20210407    LA LIBERTAD   27611   17400
13     20210407     LAMBAYEQUE   13575   12241
14     20210407           LIMA  285986  138234
15     20210407         LORETO   14578    7138
16     20210407  MADRE DE DIOS    2234    1886
17     20210407       MOQUEGUA    4267    3720
18     20210407          PASCO    3541    3274
19     20210407          PIURA   16709   12431
20     202104

In [18]:
result_query_dosis1.to_csv('../resultados/dosis1.csv')
result_query_dosis2.to_csv('../resultados/dosis2.csv')
result_query_departamentos.to_csv('../resultados/departamentos.csv')

In [19]:
print('Anna Bananna----------------------------------------------')

Anna Bananna----------------------------------------------
